# Prompt Optimization Baseline Metrics Capture

**Purpose**: Systematically capture baseline metrics before implementing prompt optimization changes.

**Created**: 2025-11-27  
**Status**: Pre-Implementation Baseline Capture

## Goals
1. Capture 10-20 keyframe prompt generations with current defaults
2. Log prompt length, token estimates, CLIP chunk counts
3. Audit feature flags for plan-to-repo drift
4. Validate negative prompt coverage and deduplication
5. Generate test scaffolds for missing test files

## Current Feature Flag Defaults (from `utils/featureFlags.ts`)
- `subjectFirstPrompts`: `false`
- `promptQualityGate`: `false`  
- `promptTokenGuard`: `'warn'` (NOT 'off' as handoff implies)
- `keyframePromptPipeline`: `true`
- `enhancedNegativePrompts`: `false`

## Section 1: Setup and Import Dependencies

In [ ]:
import os
import re
import json
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import pandas as pd

# Project paths
PROJECT_ROOT = Path(r"c:\Dev\gemDirect1")
SERVICES_DIR = PROJECT_ROOT / "services"
UTILS_DIR = PROJECT_ROOT / "utils"

# Key files to analyze
FILES = {
    "featureFlags": UTILS_DIR / "featureFlags.ts",
    "promptConstants": SERVICES_DIR / "promptConstants.ts",
    "promptPipeline": SERVICES_DIR / "promptPipeline.ts",
    "promptWeighting": SERVICES_DIR / "promptWeighting.ts",
    "tokenValidator": SERVICES_DIR / "tokenValidator.ts",
    "generationMetrics": SERVICES_DIR / "generationMetrics.ts",
    "handoff": PROJECT_ROOT / "AGENT_HANDOFF_PROMPT_OPTIMIZATION_IMPLEMENTATION_20251127.md",
    "improvementPlan": PROJECT_ROOT / "Documentation" / "PROMPT_OPTIMIZATION_IMPROVEMENT_PLAN.md",
}

# Verify all files exist
for name, path in FILES.items():
    status = "✅" if path.exists() else "❌"
    print(f"{status} {name}: {path.name}")

## Section 2: Analyze Feature Flag Configuration

Parse `utils/featureFlags.ts` to extract current defaults and compare against documentation claims.

In [ ]:
def extract_feature_flag_defaults(content: str) -> Dict[str, Any]:
    """Extract DEFAULT_FEATURE_FLAGS values from featureFlags.ts"""
    defaults = {}
    
    # Find the DEFAULT_FEATURE_FLAGS block
    match = re.search(r'export const DEFAULT_FEATURE_FLAGS.*?=\s*\{([^}]+(?:\{[^}]*\}[^}]*)*)\}', content, re.DOTALL)
    if not match:
        return defaults
    
    block = match.group(1)
    
    # Parse each flag: value line
    for line in block.split('\n'):
        line = line.strip()
        # Match patterns like: flagName: true, flagName: false, flagName: 'value'
        flag_match = re.match(r"(\w+):\s*(true|false|'[^']+'),?\s*(?://.*)?$", line)
        if flag_match:
            name = flag_match.group(1)
            value_str = flag_match.group(2)
            # Convert to Python types
            if value_str == 'true':
                value = True
            elif value_str == 'false':
                value = False
            else:
                value = value_str.strip("'")
            defaults[name] = value
    
    return defaults

# Extract actual defaults
content = FILES["featureFlags"].read_text(encoding='utf-8')
actual_defaults = extract_feature_flag_defaults(content)

# Key flags for prompt optimization
key_flags = [
    'subjectFirstPrompts',
    'promptQualityGate', 
    'promptTokenGuard',
    'keyframePromptPipeline',
    'enhancedNegativePrompts',
    'promptWeighting',
    'qualityPrefixVariant',
    'bibleV2SaveSync',
    'sceneListValidationMode',
]

print("="*60)
print("FEATURE FLAG DEFAULTS (Actual vs Handoff Claims)")
print("="*60)

# Handoff claims (from document analysis)
handoff_claims = {
    'subjectFirstPrompts': False,  # Correct in handoff
    'promptQualityGate': False,    # Correct in handoff
    'promptTokenGuard': 'off',     # WRONG - handoff says 'off', actual is 'warn'
    'keyframePromptPipeline': True, # Correct (but handoff doesn't mention)
    'enhancedNegativePrompts': False,
}

for flag in key_flags:
    actual = actual_defaults.get(flag, 'NOT FOUND')
    claimed = handoff_claims.get(flag, 'not specified')
    match = "✅" if actual == claimed else "⚠️ DRIFT"
    print(f"{match} {flag}: actual={actual}, claimed={claimed}")

## Section 3: Audit Negative Prompt Coverage

Extract `ENHANCED_NEGATIVE_SET` categories and identify missing ones.

In [ ]:
def extract_negative_categories(content: str) -> Dict[str, List[str]]:
    """Extract ENHANCED_NEGATIVE_SET categories from promptConstants.ts"""
    categories = {}
    
    # Find the ENHANCED_NEGATIVE_SET block
    match = re.search(r'export const ENHANCED_NEGATIVE_SET\s*=\s*\{([^}]+(?:\[[^\]]*\][^}]*)*)\}', content, re.DOTALL)
    if not match:
        return categories
    
    block = match.group(1)
    
    # Parse each category
    cat_pattern = r"(\w+):\s*\[((?:[^[\]]*|\[[^\]]*\])*)\]"
    for cat_match in re.finditer(cat_pattern, block, re.DOTALL):
        cat_name = cat_match.group(1)
        terms_block = cat_match.group(2)
        # Extract quoted strings
        terms = re.findall(r"'([^']+)'", terms_block)
        categories[cat_name] = terms
    
    return categories

# Extract current categories
content = FILES["promptConstants"].read_text(encoding='utf-8')
current_categories = extract_negative_categories(content)

# Required categories per plan
required_categories = {
    'quality': ['lowres', 'worst quality', 'bad quality', 'low quality', 'blurry'],  # Sample
    'anatomy': ['bad anatomy', 'deformed', 'disfigured'],  # Sample
    'composition': ['cropped', 'cut off', 'out of frame'],  # Sample
    'text_artifacts': ['text', 'watermark', 'logo', 'signature', 'username', 'copyright', 'caption', 'subtitle', 'credits', 'stock photo'],
    'depth': ['flat composition', 'no depth', 'incorrect perspective', 'fisheye distortion', 'lens distortion', 'flat lighting'],
    'motion': ['static pose', 'frozen movement', 'motion blur artifacts', 'ghosting', 'stuttering', 'temporal inconsistency', 'flickering'],
    'style_contamination': ['cartoon', 'anime', 'illustration', 'sketch', 'drawing', 'painting', 'abstract', 'surreal', 'CGI'],
    'quality_tiers': ['worst quality', 'low quality', 'normal quality', 'jpeg artifacts', 'compression artifacts', 'pixelated', 'noise', 'banding'],
}

print("="*60)
print("ENHANCED_NEGATIVE_SET CATEGORY ANALYSIS")
print("="*60)

for cat_name in required_categories:
    if cat_name in current_categories:
        count = len(current_categories[cat_name])
        print(f"✅ {cat_name}: {count} terms")
    else:
        print(f"❌ MISSING: {cat_name}")

print(f"\nCurrent categories: {len(current_categories)}")
print(f"Required categories: {len(required_categories)}")
print(f"Missing: {len(required_categories) - len(current_categories)}")

## Section 4: Validate Prompt Weighting Presets

Analyze `services/promptWeighting.ts` for existing presets and identify missing `balanced` preset.

In [ ]:
def extract_weighting_presets(content: str) -> Dict[str, Dict[str, float]]:
    """Extract WEIGHTING_PRESETS from promptWeighting.ts"""
    presets = {}
    
    # Find the WEIGHTING_PRESETS block
    match = re.search(r'export const WEIGHTING_PRESETS\s*=\s*\{([^}]+(?:\{[^}]*\}[^}]*)*)\}', content, re.DOTALL)
    if not match:
        return presets
    
    block = match.group(1)
    
    # Parse each preset
    preset_pattern = r"(\w+):\s*\{([^}]+)\}"
    for preset_match in re.finditer(preset_pattern, block):
        preset_name = preset_match.group(1)
        weights_block = preset_match.group(2)
        
        weights = {}
        for line in weights_block.split('\n'):
            weight_match = re.match(r"\s*(\w+):\s*([\d.]+)", line)
            if weight_match:
                weights[weight_match.group(1)] = float(weight_match.group(2))
        
        if weights:
            presets[preset_name] = weights
    
    return presets

# Extract current presets
content = FILES["promptWeighting"].read_text(encoding='utf-8')
current_presets = extract_weighting_presets(content)

# Required presets per plan
required_presets = {
    'subjectEmphasis': {'summary': 1.2, 'characters': 1.1, 'subject': 1.3, 'character': 1.2, 'style': 1.0, 'background': 0.8},
    'styleEmphasis': {'style': 1.1, 'technical': 1.05, 'subject': 1.0, 'character': 1.0, 'background': 0.9},
    'balanced': {'subject': 1.0, 'character': 1.0, 'style': 1.0, 'background': 1.0},  # MISSING
}

print("="*60)
print("WEIGHTING_PRESETS ANALYSIS")
print("="*60)

for preset_name in required_presets:
    if preset_name in current_presets:
        weights = current_presets[preset_name]
        print(f"✅ {preset_name}: {weights}")
    else:
        print(f"❌ MISSING: {preset_name}")

# Check for background weights (needed for V2)
has_background = any('background' in preset for preset in current_presets.values())
print(f"\n⚠️ Background weights present: {has_background}")

## Section 5: Type Contract Validation

Check `AssembledPrompt` interface alignment with `assemblePromptForProvider` return type.

In [ ]:
def extract_interface_properties(content: str, interface_name: str) -> List[str]:
    """Extract property names from a TypeScript interface"""
    properties = []
    
    pattern = rf'export interface {interface_name}\s*\{{([^}}]+)\}}'
    match = re.search(pattern, content, re.DOTALL)
    if not match:
        return properties
    
    block = match.group(1)
    
    # Match property declarations
    for line in block.split('\n'):
        prop_match = re.match(r'\s+(\w+)\??:\s*', line)
        if prop_match:
            properties.append(prop_match.group(1))
    
    return properties

def extract_function_return(content: str, func_name: str) -> List[str]:
    """Extract returned object properties from a function"""
    properties = []
    
    # Find the return statement
    pattern = rf'export function {func_name}[^{{]+\{{.*?return\s*\{{([^}}]+)\}}'
    match = re.search(pattern, content, re.DOTALL)
    if not match:
        return properties
    
    block = match.group(1)
    
    # Match property assignments
    for line in block.split('\n'):
        prop_match = re.match(r'\s+(\w+)[,:]', line)
        if prop_match:
            properties.append(prop_match.group(1))
    
    return properties

# Analyze AssembledPrompt interface
content = FILES["promptPipeline"].read_text(encoding='utf-8')
interface_props = extract_interface_properties(content, 'AssembledPrompt')
return_props = extract_function_return(content, 'assemblePromptForProvider')

print("="*60)
print("TYPE CONTRACT ANALYSIS: AssembledPrompt")
print("="*60)

print(f"\nInterface properties: {interface_props}")
print(f"Function returns: {return_props}")

# Check for tokenWarning mismatch (known issue)
has_token_warning_in_return = 'tokenWarning' in content.split('assemblePromptForProvider')[1].split('return')[1][:500]
has_token_counts_in_interface = 'tokens' in interface_props

print(f"\n⚠️ tokenWarning in return: likely present (check manually)")
print(f"tokens in interface: {has_token_counts_in_interface}")
print("\n🔧 FIX NEEDED: Align return type with interface definition")

## Section 6: Generate Baseline Metrics JSON

Create a baseline configuration and sample prompt metrics for A/B comparison.

In [ ]:
# Token estimation heuristic (matches services/tokenValidator.ts)
def estimate_tokens(text: str, provider: str = 'comfyui') -> int:
    """Estimate token count using provider-specific ratios"""
    if not text:
        return 0
    ratios = {'clip': 3.5, 'comfyui': 3.5, 'gemini': 4.0, 'default': 4.0}
    ratio = ratios.get(provider, ratios['default'])
    return int(len(text) / ratio + 0.5)

def estimate_clip_chunks(text: str) -> int:
    """Estimate CLIP 77-token chunks needed"""
    tokens = estimate_tokens(text, 'comfyui')
    return (tokens + 76) // 77  # Ceiling division

# Sample prompts representing different scene types
sample_prompts = [
    # Short scene summary
    "A dark alley at night, neon signs reflecting on wet pavement, a lone figure walks away.",
    
    # Medium complexity with characters
    "Elena, tall athletic woman with long dark hair and piercing green eyes, confronts Malachar in the ancient throne room. Dramatic shadows play across marble columns.",
    
    # Complex with style direction
    "Wide establishing shot of the cyberpunk megacity skyline at dusk. Holographic advertisements pierce the smog. Flying vehicles streak between towering arcologies. Cinematic lighting, volumetric fog, 16:9 aspect ratio.",
    
    # Character-heavy
    "Close-up on Elena's determined expression as she grips the enchanted blade. Malachar looms in the background, his glowing red eyes visible through the shadows. Golden hour lighting streams through shattered windows.",
    
    # Action scene
    "Dynamic medium shot: Elena leaps across the crumbling bridge as explosions tear through the ancient structure. Motion blur on debris, sharp focus on protagonist, epic scale composition.",
]

# Generate baseline metrics
baseline_data = {
    "captureDate": datetime.now().isoformat(),
    "featureFlagsUsed": {
        "subjectFirstPrompts": False,
        "promptQualityGate": False,
        "promptTokenGuard": "warn",
        "keyframePromptPipeline": True,
        "enhancedNegativePrompts": False,
    },
    "prompts": []
}

for i, prompt in enumerate(sample_prompts, 1):
    token_count = estimate_tokens(prompt)
    clip_chunks = estimate_clip_chunks(prompt)
    
    metrics = {
        "id": f"baseline-{i:03d}",
        "prompt": prompt,
        "charCount": len(prompt),
        "tokenEstimate": token_count,
        "clipChunks": clip_chunks,
        "subjectPosition": prompt.find(','),  # First comma marks subject end
        "withinBudget": token_count <= 600,  # sceneKeyframe budget
    }
    baseline_data["prompts"].append(metrics)
    
    print(f"Prompt {i}: {token_count} tokens, {clip_chunks} CLIP chunks, budget: {'✅' if metrics['withinBudget'] else '❌'}")

# Save baseline
baseline_path = PROJECT_ROOT / "scripts" / "baseline_metrics.json"
with open(baseline_path, 'w', encoding='utf-8') as f:
    json.dump(baseline_data, f, indent=2)
    
print(f"\n✅ Saved baseline to: {baseline_path}")

## Section 7: Generate Test Scaffold Templates

Create stub test files for missing test coverage.

In [ ]:
# Test scaffold templates
test_scaffolds = {
    "promptWeighting.test.ts": '''/**
 * Tests for promptWeighting service
 * Validates weighting syntax generation, parsing, and preset application
 */
import { describe, it, expect } from 'vitest';
import { 
    applyWeight, 
    parseWeightedTerms, 
    WEIGHTING_PRESETS,
    WEIGHT_MIN,
    WEIGHT_MAX,
} from '../promptWeighting';

describe('promptWeighting', () => {
    describe('applyWeight', () => {
        it('should return unweighted term when weight is 1.0', () => {
            expect(applyWeight('subject', 1)).toBe('subject');
            expect(applyWeight('subject', 1.0)).toBe('subject');
        });

        it('should apply weighting syntax for non-1.0 weights', () => {
            expect(applyWeight('subject', 1.2)).toBe('(subject:1.20)');
            expect(applyWeight('subject', 0.8)).toBe('(subject:0.80)');
        });

        it('should clamp weights to safe range [0.1, 2.0]', () => {
            expect(applyWeight('term', 0.05)).toBe('(term:0.10)');
            expect(applyWeight('term', 3.0)).toBe('(term:2.00)');
        });

        it('should handle empty strings', () => {
            expect(applyWeight('', 1.2)).toBe('');
            expect(applyWeight('  ', 1.2)).toBe('  ');
        });
    });

    describe('parseWeightedTerms', () => {
        it('should extract weighted terms from prompt', () => {
            const result = parseWeightedTerms('(subject:1.20), style, (background:0.80)');
            expect(result).toHaveLength(2);
            expect(result[0]).toEqual({ term: 'subject', weight: 1.20 });
            expect(result[1]).toEqual({ term: 'background', weight: 0.80 });
        });

        it('should return empty array for prompts without weights', () => {
            expect(parseWeightedTerms('subject, style, background')).toEqual([]);
        });
    });

    describe('WEIGHTING_PRESETS', () => {
        it('should have subjectEmphasis preset', () => {
            expect(WEIGHTING_PRESETS.subjectEmphasis).toBeDefined();
            expect(WEIGHTING_PRESETS.subjectEmphasis.summary).toBeGreaterThan(1);
        });

        it('should have styleEmphasis preset', () => {
            expect(WEIGHTING_PRESETS.styleEmphasis).toBeDefined();
            expect(WEIGHTING_PRESETS.styleEmphasis.style).toBeGreaterThan(1);
        });

        // TODO: Add balanced preset
        it.skip('should have balanced preset with all weights = 1.0', () => {
            expect(WEIGHTING_PRESETS.balanced).toBeDefined();
            expect(WEIGHTING_PRESETS.balanced.subject).toBe(1.0);
        });
    });
});
''',

    "styleExtractor.test.ts": '''/**
 * Tests for styleExtractor service (STUB)
 * Validates style keyword extraction from director\'s vision
 */
import { describe, it, expect } from 'vitest';

// TODO: Import from '../styleExtractor' when service is created

describe('styleExtractor', () => {
    describe('extractStyleKeywords', () => {
        it.todo('should extract lighting keywords from vision text');
        it.todo('should extract camera angle keywords');
        it.todo('should extract mood/atmosphere keywords');
        it.todo('should prioritize categories: lighting > mood > film > camera');
        it.todo('should limit to maxTerms parameter');
    });

    describe('formatStylesForPrompt', () => {
        it.todo('should format styles with weighting when enabled');
        it.todo('should format styles without weighting when disabled');
        it.todo('should handle empty style array');
    });
});
''',

    "characterTracker.test.ts": '''/**
 * Tests for characterTracker service (STUB)
 * Validates character appearance tracking and continuity analysis
 */
import { describe, it, expect } from 'vitest';

// TODO: Import from '../characterTracker' when service is created

describe('characterTracker', () => {
    describe('trackCharacterAppearances', () => {
        it.todo('should track character mentions across timeline shots');
        it.todo('should identify explicit vs implicit character references');
        it.todo('should link appearances to character profiles');
    });

    describe('analyzeCharacterContinuity', () => {
        it.todo('should warn when protagonist absent > 3 consecutive shots');
        it.todo('should warn when supporting character absent > 5 shots');
        it.todo('should not warn for normal appearance gaps');
        it.todo('should detect sudden character appearances');
    });

    describe('MAX_GAP constants', () => {
        it.todo('should define MAX_PROTAGONIST_GAP = 3');
        it.todo('should define MAX_SUPPORTING_GAP = 5');
    });
});
''',

    "generationMetrics.test.ts": '''/**
 * Tests for generationMetrics service
 * Validates A/B testing metrics collection and Bayesian analysis
 */
import { describe, it, expect } from 'vitest';
import {
    createMetrics,
    generateMetricId,
    summarizeMetricsByVariant,
    isSignificantDifference,
    MIN_SAMPLE_SIZE,
} from '../generationMetrics';

describe('generationMetrics', () => {
    describe('generateMetricId', () => {
        it('should generate unique IDs', () => {
            const id1 = generateMetricId();
            const id2 = generateMetricId();
            expect(id1).not.toBe(id2);
        });
    });

    describe('createMetrics', () => {
        it('should create metrics with required fields', () => {
            const metrics = createMetrics({
                promptVariantId: 'control',
                provider: 'comfyui',
            });
            expect(metrics.promptVariantId).toBe('control');
            expect(metrics.provider).toBe('comfyui');
            expect(metrics.timestamp).toBeDefined();
        });
    });

    describe('summarizeMetricsByVariant', () => {
        it('should aggregate metrics by variant ID', () => {
            const metrics = [
                createMetrics({ promptVariantId: 'control', provider: 'comfyui', success: true }),
                createMetrics({ promptVariantId: 'control', provider: 'comfyui', success: false }),
                createMetrics({ promptVariantId: 'optimized', provider: 'comfyui', success: true }),
            ];
            
            const summaries = summarizeMetricsByVariant(metrics);
            expect(summaries.get('control')?.sampleSize).toBe(2);
            expect(summaries.get('optimized')?.sampleSize).toBe(1);
        });
    });

    describe('isSignificantDifference', () => {
        it('should require minimum sample size', () => {
            expect(isSignificantDifference(0.5, 0.6, 10, 10)).toBe(false);
            expect(isSignificantDifference(0.5, 0.6, 30, 30)).toBe(true);
        });
    });

    describe('Bayesian A/B testing', () => {
        it.todo('should calculate P(treatment > control)');
        it.todo('should converge with fewer samples than frequentist');
    });
});
''',
}

# Check which test files exist
tests_dir = SERVICES_DIR / "__tests__"
print("="*60)
print("TEST FILE STATUS")
print("="*60)

for filename in test_scaffolds:
    filepath = tests_dir / filename
    if filepath.exists():
        print(f"✅ EXISTS: {filename}")
    else:
        print(f"❌ MISSING: {filename}")

print(f"\n📁 Test directory: {tests_dir}")

## Section 8: Documentation Drift Summary

Generate a summary of discrepancies between handoff docs and actual codebase.

In [ ]:
# Generate documentation drift report
drift_report = {
    "generated": datetime.now().isoformat(),
    "issues": [
        {
            "category": "Feature Flag Defaults",
            "issue": "promptTokenGuard default",
            "handoff_claims": "'off'",
            "actual_value": "'warn'",
            "severity": "HIGH",
            "fix": "Update handoff to reflect actual 'warn' default; Phase 6 is single-step jump to 'block'"
        },
        {
            "category": "Feature Flag Defaults",
            "issue": "keyframePromptPipeline not documented",
            "handoff_claims": "not mentioned",
            "actual_value": "true",
            "severity": "MEDIUM",
            "fix": "Add keyframePromptPipeline=true to handoff infrastructure table"
        },
        {
            "category": "Line Number References",
            "issue": "Stale line numbers throughout",
            "handoff_claims": "subjectFirstPrompts at line 78/189",
            "actual_value": "~line 73",
            "severity": "LOW",
            "fix": "Remove all line numbers; instruct agents to use symbol search"
        },
        {
            "category": "Negative Prompt Categories",
            "issue": "5 categories missing from ENHANCED_NEGATIVE_SET",
            "handoff_claims": "3/8 categories exist",
            "actual_value": "3 exist: quality, anatomy, composition",
            "severity": "HIGH",
            "fix": "Add text_artifacts, depth, motion, style_contamination, quality_tiers"
        },
        {
            "category": "Weighting Presets",
            "issue": "balanced preset missing",
            "handoff_claims": "3 presets mentioned",
            "actual_value": "2 presets exist: subjectEmphasis, styleEmphasis",
            "severity": "MEDIUM",
            "fix": "Add balanced preset with all weights = 1.0; add background weight key"
        },
        {
            "category": "Type Contract",
            "issue": "AssembledPrompt interface mismatch",
            "handoff_claims": "interface has tokens property",
            "actual_value": "assemblePromptForProvider returns tokenWarning (string), not tokens object",
            "severity": "HIGH",
            "fix": "Align return type before adding new callers"
        },
    ],
    "recommended_actions": [
        "1. Update AGENT_HANDOFF_PROMPT_OPTIMIZATION_IMPLEMENTATION_20251127.md",
        "2. Update Documentation/PROMPT_OPTIMIZATION_IMPROVEMENT_PLAN.md",
        "3. Expand ENHANCED_NEGATIVE_SET with 5 missing categories",
        "4. Add balanced preset to WEIGHTING_PRESETS",
        "5. Fix AssembledPrompt type alignment",
        "6. Create missing test scaffolds",
    ]
}

# Save drift report
drift_path = PROJECT_ROOT / "scripts" / "documentation_drift_report.json"
with open(drift_path, 'w', encoding='utf-8') as f:
    json.dump(drift_report, f, indent=2)

print("="*60)
print("DOCUMENTATION DRIFT REPORT")
print("="*60)

for issue in drift_report["issues"]:
    severity_icon = {"HIGH": "🔴", "MEDIUM": "🟡", "LOW": "🟢"}[issue["severity"]]
    print(f"\n{severity_icon} [{issue['severity']}] {issue['category']}: {issue['issue']}")
    print(f"   Claimed: {issue['handoff_claims']}")
    print(f"   Actual:  {issue['actual_value']}")
    print(f"   Fix:     {issue['fix']}")

print(f"\n✅ Saved drift report to: {drift_path}")